In [1]:
# header
import pandas as pd
import AddIPAFlex
import MatchHeadwords
from GenerateLexDir import literal_eval_col
from IPython.display import display

In [2]:
# load in df's
flexicon = pd.read_csv('FlexiconMERGE.csv', index_col='entry_id', keep_default_na=False)
senses = pd.read_csv('SensesMERGE.csv', index_col='sense_id', keep_default_na=False)

literal_eval_col(flexicon, 'sense')
literal_eval_col(flexicon, 'note')
literal_eval_col(flexicon, 'other_sources')
literal_eval_col(flexicon, 'these_vars')
literal_eval_col(flexicon, 'variant_of')

literal_eval_col(senses, 'gloss')
literal_eval_col(senses, 'def')

In [3]:
# match function expects 'ipa' key in dict parameter
# create ipa col in flexicon
bibs = [x if x else 'eppsob' for x in flexicon['bibliography']]
ipa = [AddIPAFlex.to_ipa(hdwd, bib=bib) for hdwd, bib in zip(flexicon['headword'],\
                                                         bibs)]
flexicon.loc[:, 'ipa'] = ipa

In [4]:
hdwrds = [not x for x in flexicon['variant_of']]
hdwrds = flexicon[hdwrds]
# O(thicc)
match = MatchHeadwords.match_dfs(hdwrds, hdwrds)
match.to_csv('merge_matches2_25.csv')
#match = pd.read_csv('merge_matches.csv', index_col='entry_id')
match.shape

<function match_dfs at 0x7fecf96e8cb0> 367.55195665359497


(1140, 1)

In [6]:
not_empty = [bool(x) for x in match['matches']]
match = match[not_empty]
#literal_eval_col(match, 'matches')
match.to_csv('merge_matches2_25.csv')
match.shape

(448, 1)

In [3]:
# find all indices that appear multiple times in FlexiconMERGE
# use numeric index to make my life easier
entry_ids = list(flexicon.index)
no_dupls=flexicon.reset_index()
no_dupls.loc[:,'entry_id'] = entry_ids

multiples = [x for x in entry_ids if entry_ids.count(x) > 1]
multiples = set(multiples)
        
for m in multiples:
    idcs = [i for i, x in enumerate(entry_ids) if x==m]
    orig = no_dupls.loc[idcs[0]]
    new  = no_dupls.loc[idcs[1]]
    
    for k in orig.keys():
        if k in ('other_sources', 'bibliography', 'date_modified'):
            continue
        o_val = orig[k]
        n_val = new[k]
        if not n_val:
            continue
        elif k == 'note' and len(n_val) == 1 and list(n_val.keys())[0] == 'bibliography':
            continue
        elif (not n_val) and o_val:
            continue
        elif o_val == n_val:
            continue
        else:
            print(o_val)
            print(n_val)
            display(flexicon.loc[m])
            i=None
            while i not in ('1', '2'):
                i=input()
            if i=='1':
                no_dupls = no_dupls.drop(idcs[0])
            else:
                no_dupls = no_dupls.drop(idcs[1])
            break
    else:
        # keep original by default
        no_dupls = no_dupls.drop(idcs[1])

['d86d7892-a9dd-4711-bde6-a91c478f0b29']
['d86d7892-a9dd-4711-bde6-a91c478f0b29', '201a34be-a730-4730-86eb-8fc5fe241de4']


,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
entry_id,,,,,,,,,,,
ajat_32eac6de-1dac-4fe7-96eb-afb8fdd8a693,Fieldnotes Epps/Obert; 2018,2018-07-10T15:39:39Z,,ajat,stem,{},{'Barbosa': 'ʔajat'},ʔaˈjat,[d86d7892-a9dd-4711-bde6-a91c478f0b29],{'ʔajat_c5b4a15c-8d79-4996-b956-443c2143e1b7':...,{}
ajat_32eac6de-1dac-4fe7-96eb-afb8fdd8a693,,2018-07-10T15:39:39Z,2019-12-04T23:38:53Z,ajat,stem,{'bibliography': 'Fieldnotes Epps/Obert; 2018'},{},ʔaˈjat,"[d86d7892-a9dd-4711-bde6-a91c478f0b29, 201a34b...",{},None


1

kuɾid


,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
entry_id,,,,,,,,,,,
kurid_f2e1c000-9369-44af-abbb-74551674cbcf,Fieldnotes Epps/Obert; 2018,2018-07-27T22:53:48Z,,kurid,stem,{},None,,[b6a35499-1a28-4e89-98ad-3a850f8b828a],None,{}
kurid_f2e1c000-9369-44af-abbb-74551674cbcf,,2018-07-27T22:53:48Z,2019-11-25T16:25:12Z,kurid,stem,{'bibliography': 'Fieldnotes Epps/Obert; 2018'},{},kuɾid,[b6a35499-1a28-4e89-98ad-3a850f8b828a],{},None


2
a 
a=


...

1
['df820bc8-d08d-4b87-90f0-ba34e429c0cf']
['df820bc8-d08d-4b87-90f0-ba34e429c0cf', '7c2bbda0-0a72-47f0-a8bd-1f12d7ee2ad7']


,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
entry_id,,,,,,,,,,,
tarog_881c607c-3829-44af-929d-caa7a9a1c811,Fieldnotes Epps/Obert; 2018,2019-01-14T03:46:55Z,,tarog,stem,None,None,taɾog̚,[df820bc8-d08d-4b87-90f0-ba34e429c0cf],None,{}
tarog_881c607c-3829-44af-929d-caa7a9a1c811,,2019-01-14T03:46:55Z,2019-12-15T22:57:47Z,tarog,stem,{'bibliography': 'Epps/Obert; Fieldnotes 2019'},{},taɾog̚,"[df820bc8-d08d-4b87-90f0-ba34e429c0cf, 7c2bbda...",{},None


1
['9b7058c1-6c0d-4e6f-b310-176ad6fc0dd1']
['9b7058c1-6c0d-4e6f-b310-176ad6fc0dd1', 'f048d404-610e-467b-8533-7ed6a12cb920']


,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
entry_id,,,,,,,,,,,
majoor_c276c6b7-c47e-43a9-a101-732c26dbba0d,Fieldnotes Epps/Obert; 2018,2019-01-14T03:46:55Z,,majoor,stem,None,{'Martins': 'majoːɾ'},mãjo:ɾᵊ,[9b7058c1-6c0d-4e6f-b310-176ad6fc0dd1],{'majoːɾ_736c5590-b31b-4510-b141-5baa8624fc3f'...,{}
majoor_c276c6b7-c47e-43a9-a101-732c26dbba0d,,2019-01-14T03:46:55Z,2020-01-08T15:41:47Z,majoor,stem,{'bibliography': 'Epps/Obert; Fieldnotes 2019'},{},mãjo:ɾᵊ,"[9b7058c1-6c0d-4e6f-b310-176ad6fc0dd1, f048d40...",{},None


1


In [1]:
flexicon = no_dupls
flexicon.to_csv('FlexiconMERGE.csv', index=False)
#flexicon = pd.read_csv('FlexiconMERGE.csv', index_col='entry_id', keep_default_na=False)

NameError: name 'no_dupls' is not defined

In [42]:
def find_gloss(entry_id):
    these_senses = flexicon.at[entry_id, 'sense']
    if not these_senses:
        return
    out=''
    for sense in these_senses:
        gloss = senses.at[sense, 'gloss']
        defin = senses.at[sense, 'def']
        if defin:
            out = ';'.join(gloss.values())
        elif gloss:
            out = ';'.join(gloss.values())
        
        if '?' in out:
            continue
        elif out:
            return out
    else:
        return out
    
match.loc[:, 'def'] = [find_gloss(idx) for idx in match.index]
match['def']

entry_id
h’ëëd_025fec92-6af8-4d61-8544-bdb0e0362545                                what;o que
kawa_0431db34-f253-491d-bb2e-bcb885fdcd8d          burn.big.fire;queimar.fogo.grande
asooh_045c762b-0b2e-4233-a013-7e921270d050                                sit;sentar
karapee _0608a430-217f-4595-8307-b236eef0ae5b                    child.sg;criança.sg
madyyk_09e54f3c-410b-415c-8850-75ea270e4552                           açaí.palm;açaí
                                                                 ...                
nyyd_facdf3f0-1a1e-403e-912b-c4f27a0d057e                                      WH;WH
asoop_fb0ac1ce-3487-4841-8c1e-1d831b4bedc9        go.upwards.from river;subir.do.rio
bii_fb6c7157-1b9b-4d1b-9926-dcdea3043357                                breast;seios
ëën_fe7eb662-ab08-4960-beed-46e3474784d5                          mother.Voc;mãe.Voc
jajé     _ff33ec40-692f-454d-915f-ee99245cc6f7          there.invisible;lá.invisible
Name: def, Length: 433, dtype: object

In [43]:
for row_match in match['matches']:
    for k, v in row_match.copy().items():
        v = (v[0], find_gloss(k))
        row_match[k]=v

In [44]:
match.to_csv('merge_matches.csv')